<a href="https://colab.research.google.com/github/JPalacios-8/DeepLearningFinal/blob/main/CurriculumResumenBart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROYECTO FINAL DEEP LEARNING
Extracción de características de curriculums

In [ ]:
!pip install pymupdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
import os
import fitz  #PyMuPDF
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import torch

In [ ]:
def extract_batch_texts(pdf_paths):
    rows = []
    for filepath in tqdm(pdf_paths):
        try:
            text = ""
            with fitz.open(filepath) as doc:
                for page in doc:
                    text += page.get_text()
            # Obtener categoría (nombre de subcarpeta)
            category = os.path.basename(os.path.dirname(filepath))
            filename = os.path.basename(filepath)
            rows.append({"filename": filename, "category": category, "resume_text": text})
        except Exception as e:
            print(f"Error with {filepath}: {e}")
    return pd.DataFrame(rows)


In [ ]:
# Descomprimir el ZIP
!unzip -q /content/archive.zip -d /content/

# Verifica estructura
!ls /content/data/data | head -n 10


ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT


recorre dataset y encuentra cuantos pdfs hay (curriculums)

In [ ]:
from glob import glob

pdf_folder = "/content/data/data"
pdf_files = sorted(glob(os.path.join(pdf_folder, "*/*.pdf")))
print(f"Total PDFs found: {len(pdf_files)}")


Total PDFs found: 2484


extraer los textos de los pdfs y guardarlos en la nueva carpeta: all_resume_data.csv



In [ ]:
batch_size = 500
output_path = "/content/all_resume_data.csv"
first_batch = True

for i in range(0, len(pdf_files), batch_size):
    batch_files = pdf_files[i:i + batch_size]
    print(f"📚 Processing batch {i // batch_size + 1} → Files {i} to {i + len(batch_files) - 1}")

    batch_df = extract_batch_texts(batch_files)

    batch_df.to_csv(output_path, mode='a', index=False, header=first_batch)
    first_batch = False


📚 Processing batch 1 → Files 0 to 499


100%|██████████| 500/500 [00:13<00:00, 36.85it/s] 


📚 Processing batch 2 → Files 500 to 999


100%|██████████| 500/500 [00:04<00:00, 108.42it/s]


📚 Processing batch 3 → Files 1000 to 1499


100%|██████████| 500/500 [00:05<00:00, 84.32it/s] 


📚 Processing batch 4 → Files 1500 to 1999


100%|██████████| 500/500 [00:04<00:00, 105.84it/s]


📚 Processing batch 5 → Files 2000 to 2483


100%|██████████| 484/484 [00:10<00:00, 45.41it/s]


caragr csv generado y visualizar pirmeras filas

-> contine                    1:nombre del archivo   
2: categoria  
3: todo el texto del pdf pasado a texto plano



In [ ]:


# Ruta al CSV generado previamente
csv_path = '/content/all_resume_data.csv'

# Cargar el DataFrame
data = pd.read_csv(csv_path)

# Visualizar las primeras filas
data.head()


,filename,category,resume_text
0,10554236.pdf,ACCOUNTANT,ACCOUNTANT\nSummary\nFinancial Accountant spec...
1,10674770.pdf,ACCOUNTANT,STAFF ACCOUNTANT\nSummary\nHighly analytical a...
2,11163645.pdf,ACCOUNTANT,ACCOUNTANT\nProfessional Summary\nTo obtain a ...
3,11759079.pdf,ACCOUNTANT,SENIOR ACCOUNTANT\nExperience\nCompany Name Ju...
4,12065211.pdf,ACCOUNTANT,SENIOR ACCOUNTANT\nProfessional Summary\nSenio...


Cargar modelo BART y tokenizer

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

#Cargar tokenizer y modelo BART preentrenado para resumen
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Generación de resumenes con Bart


In [ ]:

# modelo en modo evaluación
model.eval()

data = data.sample(10).reset_index(drop=True) # 10 filas aleatorias

# Generar resúmenes
generated_summaries = []

for text in tqdm(data['resume_text'], desc="Generating summaries (without fine tuning)"):
    # Tokenizar entrada
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generar resumen
    summary_ids = model.generate(
        inputs,
        max_length=150,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # Decodificar y guardar resumen
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

#Agregar columna al DataFrame
data['generated_summary'] = generated_summaries

# Mostrar una muestra
#data[['filename', 'generated_summary']].head()


Generating summaries: 100%|██████████| 5/5 [03:07<00:00, 37.58s/it]


Preparación dataset para entrenamiento

In [ ]:
# Filtrar datos válidos (por si hay vacíos)
train_data = data[
    data['resume_text'].notnull() &
    #data['auto_summary'].notnull()
    data['generated_summary'].notnull()
].reset_index(drop=True)

# Mostrar cuántos datos quedaron
print(f"Total training examples: {len(train_data)}")

# Ejemplo de un par entrada-salida
#train_data[['resume_text']].head(1) #cambiar a 5 para ver todas las del ej


Total training examples: 5


tokenizacion para posterior fine tuning
input: resume text

output (labels):generated summary

attention mask: generado a partir del modelo Bart sin fine tuning

In [ ]:

# Cargar tokenizer preentrenado de BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenizar textos de entrada (resumes)
inputs = tokenizer(
    list(train_data["resume_text"]),
    max_length=512,
    truncation=True,
    padding='max_length',
    return_tensors="pt"
)

# Tokenizar resúmenes objetivo
outputs = tokenizer(
    list(train_data["generated_summary"]),
    max_length=150,
    truncation=True,
    padding='max_length',
    return_tensors="pt"
)

# Extraer tensores
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
labels = outputs.input_ids

# Reemplazar padding tokens en labels por -100 para que no afecten el cálculo del loss
labels[labels == tokenizer.pad_token_id] = -100


dataset personalizado con pytorch: agrupa los tensores para posterior/ alimentar al modelo

In [ ]:
import torch
from torch.utils.data import Dataset

class ResumeSummaryDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }


dataloader para entrenamient

In [ ]:
from torch.utils.data import DataLoader

# Instanciar dataset
dataset = ResumeSummaryDataset(input_ids, attention_mask, labels)

# Crear DataLoader con batch]de 2
train_dataloader = DataLoader(dataset, batch_size=5, shuffle=True)


modelo de optimizacion y entrenamiento

In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from tqdm import tqdm

#modelo preentrenado
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

device = torch.device("cpu")
model = model.to(device)

# Optimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Entrenamiento
model.train()
for epoch in range(5):  # Puedes aumentar el número de épocas
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # Mover datos al dispositivo (CPU en este caso)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Adelante (forward pass)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask, #genera el valor agregado a la prediccion
            labels=labels
        )
        loss = outputs.loss

        # Atrás (backward pass)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Mostrar progreso
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 0: 100%|██████████| 3/3 [01:43<00:00, 34.36s/it, loss=0.753]


evaluacion

In [ ]:
# Asegúrate de que el modelo esté en modo evaluación
model.eval()

# Crear lista para guardar nuevos resúmenes
generated_summaries = []

# Usamos solo algunos ejemplos de prueba (ajusta si lo deseas)
test_texts = data['resume_text'].dropna().tolist()

# Generar resúmenes
for text in tqdm(test_texts, desc="Generando resúmenes"):
    # Tokenizar entrada
    inputs = tokenizer(
        text,
        return_tensors='pt',
        max_length=1024,
        truncation=True
    )

    # Generar resumen
    summary_ids = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,
        min_length=50,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Agregar nueva columna al DataFrame
data['generated_summaryFT'] = generated_summaries

# Mostrar primeros ejemplos
data[['filename', 'category', 'generated_summaryFT']].head()


Generando resúmenes: 100%|██████████| 5/5 [03:34<00:00, 42.92s/it]


,filename,category,generated_summaryFT
0,28290448.pdf,PUBLIC-RELATIONS,Communication Specialist with extensive experi...
1,22675694.pdf,DESIGNER,"Phat Le (or Danny), a high school student in F..."
2,84356308.pdf,FINANCE,The Finance Manager has 16 years of experience...
3,15602094.pdf,CONSULTANT,IT Consultant is a talented IT professional sp...
4,12839152.pdf,CONSTRUCTION,The Construction Manager is a U.S. OSHA Certif...
